### 🧩 Adım 1: Kütüphanelerin Yüklenmesi
Bu adımda Python’un standart kütüphanelerini içe aktarıyoruz.  
Bu modüller dosya yönetimi, sistem işlemleri ve metin biçimlendirme için kullanılacak.

In [1]:
import os, shutil, sys, subprocess, textwrap, json, glob, random, pathlib

### 📁 Adım 2: Proje Dizinlerinin Oluşturulması

Bu adımda, proje dosyalarının düzenli tutulabilmesi için gerekli klasör yapısı oluşturulmaktadır:

- **`PROJECT_ROOT`** → Tüm projenin kök dizini (`/content/fashion-rag-bot`)  
- **`DATA_DIR`** → Veri kümesinin (text dosyaları) tutulduğu dizin (`/content/fashion-rag-bot/data/style_guides`)  
- **`VECTOR_DIR`** → Embedding sonuçlarının ve Chroma veritabanının saklandığı dizin (`/content/fashion-rag-bot/vectordb`)

> Google Colab çalışma ortamı her yeniden başladığında bu klasörler silinir,  
> bu yüzden `os.makedirs(..., exist_ok=True)` komutu ile tekrar oluşturulur.  
> `exist_ok=True` parametresi, klasör zaten varsa hata vermemesini sağlar.




In [2]:
PROJECT_ROOT = "/content/fashion-rag-bot"
DATA_DIR     = f"{PROJECT_ROOT}/data/style_guides"
VECTOR_DIR   = f"{PROJECT_ROOT}/vectordb"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(VECTOR_DIR, exist_ok=True)

PROJECT_ROOT, DATA_DIR, VECTOR_DIR

('/content/fashion-rag-bot',
 '/content/fashion-rag-bot/data/style_guides',
 '/content/fashion-rag-bot/vectordb')

### ⚙️ Adım 3: Sistem Bağımlılıklarının Kurulması

Bu adımda, Colab çalışma ortamında bazı temel Python kütüphanelerinin belirli sürümleri kurulmaktadır.  
Amaç, farklı sürüm çakışmalarını engelleyip kodun kararlı şekilde çalışmasını sağlamaktır.

Yüklenen kütüphaneler ve görevleri:
- **datasets==2.20.0** → Hugging Face veri setleriyle çalışma imkânı sağlar.
- **langchain, langchain-community, langchain-openai** → LLM (büyük dil modelleri) ile etkileşim kurmak için kullanılır.
- **sentence-transformers==3.1.1** → Metinleri sayısal vektörlere dönüştürür (embedding işlemleri).
- **chromadb==0.5.5** → Vektör veritabanı yönetimi (örneğin arama veya benzerlik karşılaştırmaları).
- **python-dotenv==1.0.1** → .env dosyasındaki gizli API anahtarlarını yönetmek için.
- **gradio==4.39.0** → Etkileşimli web arayüzleri oluşturmak için.
- **websockets==11.0.3** → Gradio ve LangChain gibi araçların gerçek zamanlı bağlantılarını yönetir.
- **opentelemetry-*/** → Telemetri (izleme) desteği sağlar; chromadb ve langchain ile uyum için gereklidir.
- **numpy==1.26.4** → Sayısal işlemler ve matris hesaplamalarının temel kütüphanesi.

> 📌 Bu adım, Colab’da “paket sürümü uyuşmazlık hatalarını” önlemek için gereklidir.  
> Örneğin `opentelemetry` veya `requests` sürümleri farklı olursa `dependency conflict` hataları oluşabilir.

---

💡 **Kısaca bu hücre ne yapıyor?**
- `!pip -q install -U ...` → gerekli bağımlılıkları “sessiz” (quiet) şekilde yüklüyor.  
- `-U` → var olan sürümü güncelliyor (update).  
- `\` → uzun komutları birden fazla satıra bölüp okunabilir hale getiriyor.


In [3]:
!pip install -U \
  datasets==2.20.0 \
  langchain==0.3.7 \
  langchain-community==0.3.7 \
  langchain-openai==0.2.6 \
  sentence-transformers==3.1.1 \
  chromadb==0.5.5 \
  python-dotenv==1.0.1 \
  gradio==4.39.0 \
  websockets==11.0.3 \
  opentelemetry-api==1.37.0 \
  opentelemetry-sdk==1.37.0 \
  opentelemetry-exporter-otlp-proto-common==1.37.0 \
  opentelemetry-exporter-otlp-proto-http==1.37.0 \
  opentelemetry-proto==1.37.0 \
  numpy==1.26.4



### ✅ Adım 4: Kütüphanelerin Test Edilmesi

Bu adımda, bir önceki hücrede yüklenen kütüphanelerin doğru şekilde kurulduğu kontrol edilmektedir.  
Kod, gerekli modülleri içe aktarır (`import`) ve eğer herhangi bir hata oluşmazsa “Paketler hazır” mesajını ekrana yazdırır.

**Kullanılan kütüphaneler:**
- **requests** → HTTP istekleri (örneğin veri indirme işlemleri) için.  
- **datasets** → Hugging Face veri kümelerini yüklemek için.  
- **chromadb** → Metin embedding’lerini vektör veritabanına kaydetmek için.  
- **sentence_transformers.SentenceTransformer** → Cümleleri sayısal vektörlere dönüştüren model sınıfı.  




In [4]:
import requests, datasets, chromadb
from sentence_transformers import SentenceTransformer
print("✅ Paketler hazır")


✅ Paketler hazır


### 🔑 Adım 5: Gemini API Anahtarının Tanımlanması

Bu adımda, Google’ın **Gemini** (Generative AI) modeline erişmek için gerekli olan API anahtarı sisteme tanıtılır.  
Amaç, kodun daha sonraki adımlarda Google Gemini modellerini (örneğin `gemini-2.5-flash`) kullanabilmesini sağlamaktır.

---

#### 🧩 Kod Açıklaması

- `!pip -q install google-generativeai langchain-google-genai`  
  → Gemini modelini ve LangChain’in bu modele bağlanmasını sağlayan kütüphaneler yüklenir.  

- `import os, getpass`  
  → `os` modülü ortam değişkenleriyle çalışmak,  
    `getpass` ise API anahtarını gizli (ekranda görünmeden) kullanıcıdan almak için kullanılır.  

- `os.environ["EMBED_MODEL"]`  
  → Embedding (metinleri sayısal vektörlere dönüştürmek) için kullanılacak model belirlenir:  
    `"sentence-transformers/all-MiniLM-L6-v2"`.  

- `os.environ["CHROMA_DIR"]`  
  → Chroma vektör veritabanının kayıt dizini belirlenir:  
    `"/content/fashion-rag-bot/vectordb"`.  

- `key = getpass.getpass("Gemini API Key: ")`  
  → Kullanıcıdan **Gemini API anahtarı** gizli olarak alınır.  

- `os.environ["GEMINI_API_KEY"] = key`  
  → Bazı kütüphaneler bu isimde anahtarı arar.  

- `os.environ["GOOGLE_API_KEY"] = key`  
  → `langchain-google-genai` kütüphanesi bu ortam değişkenini zorunlu olarak ister.  

- `os.environ.pop("GOOGLE_APPLICATION_CREDENTIALS", None)` ve `os.environ.pop("GOOGLE_CLOUD_PROJECT", None)`  
  → Google Cloud’un varsayılan kimlik bilgilerini temizler. (Yanlışlıkla başka proje kimlikleri karışmasın diye.)  

---

#### 💡 Kısaca Ne Yapar?
Bu hücre:
1. Gerekli Gemini bağlantı kütüphanelerini yükler,  
2. API anahtarını güvenli şekilde alır,  
3. Anahtarı Colab ortamına kaydeder,  
4. Gereksiz Google kimlik değişkenlerini temizler.  

✅ **Sonuç:**  
Gemini modellerine bağlanmak için ortam değişkenleri başarıyla ayarlanır.  
Ekranda “✅ API key ortam değişkenleri ayarlandı.” mesajı görülür.


In [5]:
!pip -q install google-generativeai langchain-google-genai
import os, getpass

os.environ["EMBED_MODEL"] = "sentence-transformers/all-MiniLM-L6-v2"
os.environ["CHROMA_DIR"] = "/content/fashion-rag-bot/vectordb"
key = getpass.getpass("Gemini API Key: ")
os.environ["GEMINI_API_KEY"] = key
os.environ["GOOGLE_API_KEY"] = key


os.environ.pop("GOOGLE_APPLICATION_CREDENTIALS", None)
os.environ.pop("GOOGLE_CLOUD_PROJECT", None)

print("✅ API key ortam değişkenleri ayarlandı.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
Gemini API Key: ··········
✅ API key ortam değişkenleri ayarlandı.


### 🧠 Adım 6 : Gemini Modellerinin Test Edilmesi





Bu hücre, API bağlantısının doğru kurulduğunu ve hangi Gemini modellerine erişim sağlandığını test eder.

---

#### 🧩 Kod Açıklaması

- `import google.generativeai as genai, os`  
  → Google’ın resmi `generativeai` kütüphanesi içe aktarılır.  
  → `os` modülü, ortam değişkenlerinden (örneğin API key) bilgi çekmek için kullanılır.  

- `genai.configure(api_key=os.environ["GOOGLE_API_KEY"])`  
  → Bir önceki adımda tanımlanan **Gemini API anahtarı** kullanılarak bağlantı yapılandırılır.  

- `for m in genai.list_models():`  
  → Hesabının erişebildiği tüm Gemini modelleri döngüyle listelenir.  

- `print(m.name, m.supported_generation_methods)`  
  → Her modelin adını (`m.name`) ve hangi tür işlemleri desteklediğini (`m.supported_generation_methods`) yazdırır.  
    Örneğin:
    ```
    models/gemini-2.5-flash ['generateContent', 'countTokens']
    models/gemini-2.5-pro ['generateContent', 'batchGenerateContent']
    ```

---

#### 💡 Kısaca Ne Yapar?
- Gemini API’nin doğru çalışıp çalışmadığını kontrol eder.  
- Hesabında erişim bulunan tüm modelleri listeler.  
- Hangi modelin “generateContent”, “embedText” veya “countTokens” işlemlerini desteklediğini gösterir.






In [6]:
import google.generativeai as genai, os
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
for m in genai.list_models():
    print(m.name, m.supported_generation_methods)


models/embedding-gecko-001 ['embedText', 'countTextTokens']
models/gemini-2.5-pro-preview-03-25 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-flash-preview-05-20 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-flash ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-flash-lite-preview-06-17 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro-preview-05-06 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro-preview-06-05 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash ['generateContent', '

## 👗 Adım 7: Veri Kümesinin İndirilmesi ve Metin Dosyalarına Dönüştürülmesi

Bu adımda, proje için kullanılacak olan **fashion-style-instruct** veri kümesi **Hugging Face** üzerinden indirilir  
ve her bir örnek `.txt` dosyasına dönüştürülür.  
Bu metin dosyaları, sonraki adımda **Chroma vektör veritabanı** oluşturulurken kullanılacaktır.

---

### 🧩 Teknik Açıdan Kodun Açıklaması

#### 1️⃣ Kütüphanelerin Yüklenmesi
```python
from datasets import load_dataset
import os, glob
```
 - `datasets (Hugging Face)`  
   → Açık kaynak veri kümelerini indirip yüklemek için.
 - `os, glob`

   → Dosya yollarını yönetmek ve .txt dosyalarını işlemek için.
---
####2️⃣ Çıktı Dizininin Oluşturulması
```python
OUT_DIR = "/content/fashion-rag-bot/data/style_guides"
os.makedirs(OUT_DIR, exist_ok=True)
```
 - `OUT_DIR`

   → .txt dosyalarının kaydedileceği dizin.

 - `os.makedirs(..., exist_ok=True)`

   → Klasör yoksa oluşturur, varsa hata vermez.

💡 Bu klasör, her bir moda önerisinin ayrı bir metin dosyası olarak kaydedileceği yerdir.

---
####3️⃣ Veri Kümesinin Yüklenmesi
```python
ds = load_dataset("neuralwork/fashion-style-instruct", split="train")
```
- ` "neuralwork/fashion-style-instruct" `

   → Moda tarzı önerileri içeren bir “instruction-based” veri kümesidir.

- ` split="train" `

   → Veri kümesinin eğitim kısmını (train set) indirir.

Veri kümesindeki her satır, üç ana sütundan oluşur:
- `input`
  → Kullanıcının tarz profili (örneğin: “Spor giyim tarzını seven bir kadın”)

- `context`
  → Kombin önerisinin yapılacağı durum (örneğin: “Kış daveti için kombin öner”)

- `completion`
  → Yapay zekânın oluşturduğu kombin önerisi
---

#### 4️⃣ row_to_text() Fonksiyonunun Görevi
```python
def row_to_text(row):
    input_txt = (row.get("input") or "").strip()
    context   = (row.get("context") or "").strip()
    completion= (row.get("completion") or "").strip()
    return f"USER_PROFILE: {input_txt}\nOCCASION: {context}\n\nRECOMMENDATION:\n{completion}\n"
```
- Bu fonksiyon, veri kümesindeki her satırı düzenli ve okunabilir bir metin formatına dönüştürür.
   - 📄 Örnek çıktı:
        - USER_PROFILE: Klasik giyim tarzını seven bir kadın
        - OCCASION: Yaz daveti için kombin önerisi
        - RECOMMENDATION:İnce askılı midi elbise, hasır çanta ve açık topuklu sandalet önerilir.
    - Bu format sayesinde model, her moda önerisini “kullanıcı profili → bağlam → öneri” şeklinde net biçimde öğrenir.
Ayrıca bu yapı, embedding (vektörleştirme) aşamasında modelin anlam ilişkilerini daha iyi kurmasını sağlar.
---
#### 5️⃣ Satırların .txt Dosyalarına Dönüştürülmesi
```python
for i, row in enumerate(ds):
    with open(os.path.join(OUT_DIR, f"style_{i:05d}.txt"), "w", encoding="utf-8") as f:
        f.write(row_to_text(row))

```
- enumerate(ds)

  → her satıra sıra numarası verir (0, 1, 2, …).

- f"style_{i:05d}.txt"

  → dosya adını sayıya göre oluşturur:
  style_00001.txt, style_00002.txt, style_00003.txt …

- utf-8

  → Türkçe karakterlerin doğru kaydedilmesini sağlar.

- row_to_text(row)

  → her satırı biçimlendirilmiş metin olarak dosyaya yazar.
---
#### 6️⃣ İşlem Sonucu Bilgisi
```python
print("✅ Dataset metin dosyalarına dönüştürüldü:", len(glob.glob(os.path.join(OUT_DIR, '*.txt'))))
```
- Tüm oluşturulan .txt dosyalarını sayar ve toplam miktarı ekrana yazdırır.
---

#### 💡 Kısaca Ne Yapar?
- Veri kümesini Hugging Face’ten indirir.
- Her bir satırı anlamlı metin biçimine dönüştürür
- Her metni ayrı .txt dosyası olarak kaydeder.
- Toplam kaç dosya oluşturulduğunu ekrana yazdırır.

In [7]:
from datasets import load_dataset
import os, glob

OUT_DIR = "/content/fashion-rag-bot/data/style_guides"
os.makedirs(OUT_DIR, exist_ok=True)

ds = load_dataset("neuralwork/fashion-style-instruct", split="train")

def row_to_text(row):
    input_txt = (row.get("input") or "").strip()
    context   = (row.get("context") or "").strip()
    completion= (row.get("completion") or "").strip()
    return f"USER_PROFILE: {input_txt}\nOCCASION: {context}\n\nRECOMMENDATION:\n{completion}\n"

for i, row in enumerate(ds):
    with open(os.path.join(OUT_DIR, f"style_{i:05d}.txt"), "w", encoding="utf-8") as f:
        f.write(row_to_text(row))
print("✅ Dataset metin dosyalarına dönüştürüldü:", len(glob.glob(os.path.join(OUT_DIR, '*.txt'))))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3193 [00:00<?, ? examples/s]

✅ Dataset metin dosyalarına dönüştürüldü: 3193


## 🧠 Adım 8: Embedding ve Chroma Vektör Veritabanının Oluşturulması

Bu adımda, bir önceki aşamada `.txt` formatına dönüştürülen moda önerisi metinleri,  
**sayısal vektörlere (embeddings)** dönüştürülür ve **Chroma** veritabanına kaydedilir.  

Amaç, her moda önerisini anlam temelli olarak temsil edebilmek ve  
kullanıcının sorduğu sorulara en uygun önerileri **benzerlik araması (semantic search)** ile getirebilmektir.

---

### 🧩 1️⃣ Kütüphanelerin Yüklenmesi

```python
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
```
- TextLoader

  → .txt dosyalarını okuyarak belge haline getirir.

- RecursiveCharacterTextSplitter

  → Uzun metinleri küçük parçalara (chunk) ayırır.

- Chroma

  → Metin embedding’lerini saklayan bir vektör veritabanıdır.

- HuggingFaceEmbeddings

  → Hugging Face modelini kullanarak metinleri sayısal vektörlere dönüştürür.
---
#### 📂 2️⃣ Dizin ve Model Ayarları
```python
DATA_DIR = "/content/fashion-rag-bot/data/style_guides"
CHROMA_DIR = os.environ["CHROMA_DIR"]
EMBED_MODEL = os.environ["EMBED_MODEL"]
```
- DATA_DIR

  → .txt moda önerilerinin bulunduğu klasör.

- CHROMA_DIR

  → Oluşturulan Chroma veritabanının kaydedileceği dizin.

- EMBED_MODEL

  → Embedding modeli olarak "sentence-transformers/all-MiniLM-L6-v2" kullanılır.
---
#### 📄 3️⃣ Tüm Metin Dosyalarının Yüklenmesi
```python
docs = []
for path in glob.glob(os.path.join(DATA_DIR, "*.txt")):
    docs.extend(TextLoader(path, encoding="utf-8").load())
```
- glob.glob()

  → Klasördeki tüm .txt dosyalarını bulur.

- TextLoader()

  → Dosyaları okur ve “document” formatına dönüştürür.

- Sonuç olarak, tüm belgeler docs listesinde toplanır.

💡 Türkçe karakterlerin bozulmaması için utf-8 kodlaması kullanılır.

---
#### ✂️ 4️⃣ Metinlerin Parçalara Ayrılması (Chunking)
```python
splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=80)
chunks = splitter.split_documents(docs)
```
- Uzun metinleri 600 karakterlik parçalara böler.

- Her parçanın son 80 karakteri, bir sonrakiyle örtüşür (overlap) — bu, anlam bütünlüğünü korur.
- Örneğin:
  ```python
  [Parça 1: 0–600], [Parça 2: 520–1120], [Parça 3: 1040–1640] ...
  ```
  - Böylece model metinleri bağlamını kaybetmeden işler.
---
#### 🔢 5️⃣ Embedding Modelinin Oluşturulması
```python
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
```
- Bu model, her metin parçasını sayısal vektörlere dönüştürür.
Yani metin artık sayılarla ifade edilir, bu sayılar metnin anlamını taşır.

```python
“beyaz gömlek” → [0.12, -0.44, 0.89, ...]
“beyaz tişört” → [0.10, -0.47, 0.91, ...]
```
  - Bu iki vektör birbirine yakın olduğu için, model bu iki ifadenin benzer olduğunu anlar.
---
#### 🧱 6️⃣ Chroma Vektör Veritabanının Oluşturulması
```python
vectordb = Chroma.from_documents(chunks, embedding=embeddings, persist_directory=CHROMA_DIR)
vectordb.persist()
print("✅ Chroma veritabanı oluşturuldu:", CHROMA_DIR)
```
- Chroma.from_documents()

  → Tüm embedding’leri veritabanına kaydeder.

- persist_directory

  → Verilerin kalıcı olarak saklanacağı klasör.

- vectordb.persist()

  → Veritabanını diske yazar (Colab oturumu kapansa bile kalır).

- print()

  → İşlemin başarıyla tamamlandığını bildirir.

---
#### 💡 Kısaca Ne Yapar?
- .txt moda önerisi dosyalarını okur.
- Uzun metinleri küçük parçalara böler.
- Metin parçalarını embedding modeline gönderip sayısal vektörlere çevirir.
- Bu vektörleri Chroma veritabanına kaydeder.


In [8]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

DATA_DIR = "/content/fashion-rag-bot/data/style_guides"
CHROMA_DIR = os.environ["CHROMA_DIR"]
EMBED_MODEL = os.environ["EMBED_MODEL"]

docs = []
for path in glob.glob(os.path.join(DATA_DIR, "*.txt")):
    docs.extend(TextLoader(path, encoding="utf-8").load())

splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=80)
chunks = splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
vectordb = Chroma.from_documents(chunks, embedding=embeddings, persist_directory=CHROMA_DIR)
vectordb.persist()
print("✅ Chroma veritabanı oluşturuldu:", CHROMA_DIR)


/tmp/ipython-input-3886295613.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


✅ Chroma veritabanı oluşturuldu: /content/fashion-rag-bot/vectordb


/tmp/ipython-input-3886295613.py:19: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


## 🤖 Adım 10: Gemini RAG (Retrieval-Augmented Generation) Zincirinin Kurulması

Bu adımda, daha önce oluşturulan **Chroma vektör veritabanı** ile **Google Gemini modelini** birleştirerek  
“Retrieval-Augmented Generation (RAG)” mimarisi kurulmaktadır.  

Bu sayede chatbot, kullanıcıdan gelen sorulara **veri tabanındaki gerçek bilgilere dayanarak** yanıt verebilir.

---

### 🧩 1️⃣ Kütüphanelerin Yüklenmesi

```python
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
import os, glob
```
- ChatGoogleGenerativeAI

  → Gemini modellerini LangChain içinde kullanmayı sağlar.

- Chroma

  → Vektör veritabanı (önceden oluşturulmuş moda öneri embedding’lerini saklar).

- HuggingFaceEmbeddings

  → Aynı embedding modeliyle sorguları vektörleştirir.

- ChatPromptTemplate

  → LLM’e (dil modeline) iletilecek sistem ve kullanıcı mesajlarını düzenler.

- RunnablePassthrough

  → Zincirde doğrudan kullanıcı girdisini bir sonraki aşamaya aktarmak için kullanılır.
---
#### 💬 2️⃣ Sistem Mesajı ve Prompt Tasarımı
```python
SYSTEM = """You are a professional fashion stylist.
Answer briefly and clearly in Turkish.
Use ONLY the provided CONTEXT. If unsure, say you are not sure.
Add a 'Kaynaklar:' list with file names used.
"""
PROMPT = ChatPromptTemplate.from_messages([
    ("system", SYSTEM),
    ("human", "Soru: {question}\n\nCONTEXT:\n{context}")
])

```
 - Amaç: Gemini’ye hangi rolü üstleneceğini ve nasıl cevap vermesi gerektiğini belirtmek.

- Model, yalnızca sağlanan bağlam (context) içindeki bilgilere dayanarak yanıt verir.

- Yanıt sonunda, kullandığı .txt kaynak dosyalarının isimlerini “Kaynaklar:” altında listeler.

- Türkçe yanıt verir, emin değilse “emin değilim” şeklinde belirtir.

    - 📘 Örnek prompt yapısı:
    ```python
         Soru: Kum saati vücut tipine göre yaz kombini öner.
          CONTEXT:
          [style_00023.txt] Kullanıcı profili: ...
          [style_00078.txt] Tavsiye edilen kombinler: ...
     ```
---

#### 🧾 3️⃣ Belgelerin Biçimlendirilmesi

 ``` python
  def format_docs(docs):
    import os
    return "\n\n".join([f"[{os.path.basename(d.metadata.get('source',''))}] {d.page_content}" for d in docs])
 ```
 - Bu fonksiyon, Chroma’dan dönen belgeleri formatlar.

- Her belgenin başına kaynak dosya adı (style_00001.txt) eklenir.

- Ardından metin içeriği (page_content) birleştirilerek LLM’e bağlam olarak gönderilir.
---
#### 🧱 4️⃣ Veritabanı ve Embedding Modelinin Tanımlanması
 ``` python
CHROMA_DIR = os.getenv("CHROMA_DIR", "/content/fashion-rag-bot/vectordb")
emb = HuggingFaceEmbeddings(model_name=os.getenv("EMBED_MODEL","sentence-transformers/all-MiniLM-L6-v2"))
db  = Chroma(persist_directory=CHROMA_DIR, embedding_function=emb)
retriever = db.as_retriever(search_kwargs={"k": 4})
 ```
 - CHROMA_DIR

  → Vektör veritabanının bulunduğu dizin.

- HuggingFaceEmbeddings

  → Aynı model, hem veritabanını hem de sorguları sayısallaştırır.

- retriever

  → Sorgu ile en çok benzerlik gösteren 4 belgeyi (k=4) getirir.

💡 Bu adımda, kullanıcı “Yaz daveti için kombin öner” dediğinde, Chroma veritabanından
en ilgili 4 moda önerisi metni alınır ve Gemini’ye bağlam olarak gönderilir.

---
#### ⚙️ 5️⃣ Gemini Modelinin Oluşturulması
 ``` python
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2,
    google_api_key=os.environ["GEMINI_API_KEY"]
)
```
- model

  → “gemini-2.5-flash” hızlı ve maliyet etkin bir modeldir.

- temperature=0.2

  → Daha kararlı ve tutarlı cevaplar üretmesi için düşük sıcaklık değeri.

- google_api_key

   → Google AI Studio’dan alınan Gemini API anahtarı.
---
#### 🔗 6️⃣ RAG Zincirinin Kurulması
``` python
chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} | PROMPT | llm)
print("✅ Gemini RAG zinciri hazır")
```
- retriever → format_docs

  → Sorguya uygun belgeleri getirip biçimlendirir.

- PROMPT

   → Sistem ve kullanıcı mesajlarını düzenler.

- llm

  → Gemini modeline iletir ve cevabı oluşturur.

    -   Bu yapı, LangChain “Runnable” zincir mimarisidir.
   Her adım bir öncekinin çıktısını otomatik olarak bir sonrakine aktarır.

---
#### 💡 Kısaca Ne Yapar?
- Sistem mesajı ile Gemini’ye rol (moda stilisti) atanır.
- Kullanıcı sorusu ve bağlam metinleri birleştirilir.
- Chroma veritabanından en uygun belgeler alınır.
- Gemini, sadece bu belgeleri kullanarak yanıt üretir.
- Yanıt sonunda kullanılan kaynak dosyalar listelenir.


In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
import os, glob

SYSTEM = """You are a professional fashion stylist.
Answer briefly and clearly in Turkish.
Use ONLY the provided CONTEXT. If unsure, say you are not sure.
Add a 'Kaynaklar:' list with file names used.
"""
PROMPT = ChatPromptTemplate.from_messages([
    ("system", SYSTEM),
    ("human", "Soru: {question}\n\nCONTEXT:\n{context}")
])

def format_docs(docs):
    import os
    return "\n\n".join([f"[{os.path.basename(d.metadata.get('source',''))}] {d.page_content}" for d in docs])

CHROMA_DIR = os.getenv("CHROMA_DIR", "/content/fashion-rag-bot/vectordb")
emb = HuggingFaceEmbeddings(model_name=os.getenv("EMBED_MODEL","sentence-transformers/all-MiniLM-L6-v2"))
db  = Chroma(persist_directory=CHROMA_DIR, embedding_function=emb)
retriever = db.as_retriever(search_kwargs={"k": 4})


llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2,
    google_api_key=os.environ["GEMINI_API_KEY"]
)

chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} | PROMPT | llm)
print("✅ Gemini RAG zinciri hazır")


/tmp/ipython-input-2619111312.py:24: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db  = Chroma(persist_directory=CHROMA_DIR, embedding_function=emb)
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


✅ Gemini RAG zinciri hazır


## 💬 Adım 11: Gradio ile Etkileşimli Chatbot Arayüzünün Oluşturulması

Bu adımda, **Gradio** kullanılarak kullanıcıların RAG tabanlı moda stilisti botuyla etkileşime geçebileceği  
görsel bir arayüz (UI) oluşturulmaktadır.  

Artık kullanıcı, bir moda sorusu yazarak Gemini modelinden **anlam temelli, bağlama dayalı** yanıt alabilir.

---

### 🧩 1️⃣ Gerekli Kütüphanenin Yüklenmesi

```python
import gradio as gr
```
- Gradio

  → Python projelerini interaktif web arayüzlerine dönüştürmek için kullanılan bir kütüphanedir.
  Kullanıcı, tarayıcı üzerinden sorular sorabilir ve modelin yanıtlarını canlı olarak görebilir.
---
#### 💬 2️⃣ Chatbot Yanıt Fonksiyonunun Tanımlanması
```python
def ask_bot(q):
    if not q.strip():
        return "❗ Lütfen bir soru yazın."
    ans = chain.invoke(q)
    return ans.content
```
- ask_bot(q)

  → Kullanıcıdan gelen girdiyi (q) alır ve RAG zinciri (chain) üzerinden modele gönderir.

- Boş kontrolü

  → Eğer kullanıcı herhangi bir şey yazmazsa uyarı verir: "❗ Lütfen bir soru yazın."

- chain.invoke(q)

  → Daha önce oluşturulan Gemini RAG zincirini çağırır ve modeli çalıştırır.

- ans.content

  → Modelden dönen yanıtı metin biçiminde döndürür.

---
#### 🧱 3️⃣ Gradio Arayüzünün Oluşturulması
```python
demo = gr.Interface(
    fn=ask_bot,
    inputs=gr.Textbox(lines=2, placeholder="Örn: 'Kum saati vücut tipine göre yaz kombini öner'", label="Sorunuzu yazın"),
    outputs=gr.Markdown(label="Yanıt"),
    title="🧥 Fashion RAG Chatbot (Gemini)",
    description="Bu chatbot, moda tarzı önerilerini neuralwork/fashion-style-instruct veri kümesinden öğrenerek RAG mimarisiyle üretir."
)
```
🔹 Parametreler:
| Parametre       | Açıklama                                                                                                                                    |
| --------------- | ------------------------------------------------------------------------------------------------------------------------------------------- |
| **fn**          | Kullanıcıdan alınan girdiyi işleyecek fonksiyon (`ask_bot`).                                                                                |
| **inputs**      | Kullanıcının yazacağı metin kutusu. <br><br> `lines=2` → 2 satırlık metin alanı <br><br>`placeholder` → örnek metin <br><br> `label` → kutunun başlığı |
| **outputs**     | Modelin cevabını Markdown formatında gösterir.                                                                                              |
| **title**       | Uygulamanın başlığı olarak görünür (“Fashion RAG Chatbot”).                                                                                 |
| **description** | Kullanıcıya chatbotun ne yaptığını açıklar.                                                                                                 |

----
#### 🚀 4️⃣ Arayüzün Başlatılması
```python
demo.launch(share=True)
```
- launch()

  → Gradio uygulamasını çalıştırır.

- share=True

  → Colab üzerinde geçici bir genel bağlantı (public URL) oluşturur.
   Böylece chatbot’u diğer kullanıcılarla paylaşabilir veya test edebilirsin.

📎 Örnek bağlantı çıktısı:
```python
Running on public URL: https://12345abcd.gradio.live

```
---
#### 🧾 5️⃣ Kullanım Akışı
- 1️⃣ Kullanıcı, “Sorunuzu yazın” alanına bir soru yazar:

      “Kum saati vücut tipine göre yaz kombini öner.”

- 2️⃣ ask_bot() fonksiyonu bu girdiyi Gemini RAG zincirine gönderir.
- 3️⃣ RAG zinciri, Chroma vektör veritabanından en ilgili belgeleri bulur.
- 4️⃣ Gemini modeli, bu belgelerden anlamlı ve Türkçe bir yanıt üretir.
- 5️⃣ Yanıt, Gradio arayüzünde Markdown formatında görüntülenir.
---

#### 💡 Kısaca Ne Yapar?
- Gradio kütüphanesini yükler.
- Kullanıcıdan alınan soruyu modele ileten ask_bot() fonksiyonunu tanımlar.
- Etkileşimli bir arayüz (input-output sistemi) kurar.
- Chatbot’u web arayüzü olarak başlatır ve paylaşılabilir bağlantı oluşturur.

In [10]:
import gradio as gr

def ask_bot(q):
    if not q.strip():
        return "❗ Lütfen bir soru yazın."
    ans = chain.invoke(q)
    return ans.content

demo = gr.Interface(
    fn=ask_bot,
    inputs=gr.Textbox(lines=2, placeholder="Örn: 'Kum saati vücut tipine göre yaz kombini öner'", label="Sorunuzu yazın"),
    outputs=gr.Markdown(label="Yanıt"),
    title="🧥 Fashion RAG Chatbot (Gemini)",
    description="Bu chatbot, moda tarzı önerilerini neuralwork/fashion-style-instruct veri kümesinden öğrenerek RAG mimarisiyle üretir."
)

demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.39.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://55ccc9518a5b5308db.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
